# Launching a GPU instance on KVM@TACC

The first thing we'll have to do is configure our experiment environment to use KVM.

In [ ]:
from chi import context

context.use_site("KVM@TACC")
context.choose_project()

#### Listing flavors
To get the list of reservable flavors, you can filter the output of `server.list_flavors`. We'll need the ID of this flavor to make a lease.

In [ ]:
import chi.server

flavors = chi.server.list_flavors(reservable=True)
# filter flavors by the one we are looking for
my_flavor = next(f for f in flavors if f.name == "g1.h100.pci.1")
print(f"{my_flavor.name} --- {my_flavor.description}")

In [ ]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-gpu-lease", duration=timedelta(hours=2))
my_lease.add_flavor_reservation(id=my_flavor.id, amount=1)
my_lease.submit(idempotent=True)

### Creating a server

Now you can get the flavor from your lease, and use it when creating a server. Unlike baremetal, starting a VM only takes 1-3 minutes.

In [ ]:
from chi import server

reserved_flavor = my_lease.get_reserved_flavors().pop()

my_server = server.Server(
    f"{os.getenv('USER')}-gpu-instance",
    flavor_name=reserved_flavor.name,
    image_name="CC-Ubuntu24.04-CUDA",
    key_name="mark-work",
)
my_server.submit(idempotent=True)

#### Associating a floating IP address

There are no floating IP reservations at KVM@TACC, we can just grab a floating IP address from the pool of available floating IPs. It is important to remember that we only have a finite amount of floating IP addresses, and it is critical to your fellow researchers that you are conservative in the number you allocate to your own experiments.

If you need multiple VMs for your experiment, consider putting them all on one network. Then, you can use one floating IP to connect to a "head" node, and connect to all your other nodes via the head node.

In [ ]:
fip = my_server.associate_floating_ip()

#### Security Groups
A unique feature of the KVM cloud is availability of security groups. These are unique firewall rules that are configurable via OpenStack and the horizon dashboard. This is a convenient way of configuring your VM's security. These groups exist on the bare-metal cloud, but they don't actually do anything there.

By default, all incoming connections to your VM are blocked. In order to allow remote connections, we'll need to assign the "Allow SSH" security group to our VM. We'll find it by listing all the groups below.

In most cases, _this is the ONLY security group you should ever add to your VM._

In [ ]:
from chi import network

existing_groups = network.list_security_groups(name_filter="ssh")
if existing_groups:
    sg = existing_groups.pop()
    print(f"Using existing SSH security group - '{sg.name}'")
else:
    print("Creating new SSH group 'Allow SSH'")
    sg = network.SecurityGroup({"name": "Allow SSH", "description": "Allow incoming SSH connections"})
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()

my_server.add_security_group(sg.id)

In [ ]:
my_server.check_connectivity()

## Using the GPU

Since we launched a CUDA image, we can confirm the GPU is working by running `nvidia-smi`. This will output information about the GPU such as the model and the driver.

In [ ]:
my_server.execute("nvidia-smi", connect_kwargs={"key_filename": "/Users/mark/.ssh/id_rsa"})

Now you can do anything you want that normally requires a GPU!

# (Optional) Add a persistant volume

Your instance will be deleted when your lease ends. One advantage of using KVM is that you can attach persistant volumes to your instances as a block device, and then mount it. Next time you launch an instance, you can reattach the same volume to resume your work.

First, we'll create a new volume.

In [ ]:
from chi import storage

v = storage.Volume(f"{os.getenv('USER')}-gpu-volume-2", 1)  # 1 GB in size.
v.submit(idempotent=True)

Next, attach it to the instance.

In [ ]:
my_server.attach_volume(v.id)

Now you should see 2 devices under `lsblk`. The first will be the root disk that the OS is installed on. The second will be a 1GB sized disk. To mount it, see the man page or an online guide for the `mount` command.

In [ ]:
my_server.execute("lsblk", connect_kwargs={"key_filename": "/Users/mark/.ssh/id_rsa"})

To cleanup, we can detach the volume from the server. If you don't want to keep your data, you can uncommend the line that also deletes your volume.

In [ ]:
my_server.detach_volume(v.id)
v.delete()